In [1]:
import pandas as pd
import re

In [2]:
# importing cleaned data + brands.csv
orders_cl = pd.read_parquet("orders_cl.parquet")
orderlines_cl = pd.read_parquet("orderlines_cl.parquet")
products_cl = pd.read_parquet("products_cl.parquet")
brands = pd.read_csv("brands.csv")

In [3]:
# define display format
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_rows', 1000)

    orders –> Keep only orders with the states that you want to work with.
    orderlines –> Keep only orders present in orders.
    orders –> Keep only orders present in orderlines.

In [4]:
# Exclude unwanted orders
orders_cl.groupby("state").count()

,order_id,created_date,total_paid
state,,,
Cancelled,7233,7233,7233
Completed,46605,46605,46605
Pending,14374,14374,14374
Place Order,40883,40883,40883
Shopping Basket,117809,117809,117809


In [5]:
orders_cl=orders_cl.loc[orders_cl['state']=='Completed']

In [6]:
# fixing datatypes
orders_cl.loc[:,"created_date"] = pd.to_datetime(orders_cl.loc[:,"created_date"])

In [7]:
# orderlines –> Keep only orders present in orders
orders_order_ids = orders_cl["order_id"]
orderlines_cl = orderlines_cl[orderlines_cl["id_order"].isin(orders_order_ids)].copy()
orderlines_cl

,id,id_order,product_id,product_quantity,sku,unit_price,date
7,1119116,299545,0,1,OWC0100,47.49,2017-01-01 01:46:16
8,1119119,299546,0,1,IOT0014,18.99,2017-01-01 01:50:34
9,1119120,295347,0,1,APP0700,72.19,2017-01-01 01:54:11
11,1119126,299549,0,1,PAC0929,2565.99,2017-01-01 02:07:42
18,1119134,299556,0,1,CRU0039-A,60.90,2017-01-01 02:20:14
...,...,...,...,...,...,...,...
293599,1649474,525664,0,1,TUC0207,16.52,2018-03-14 11:45:05
293616,1649512,527070,0,2,APP0698,9.99,2018-03-14 11:49:01
293621,1649522,527074,0,2,APP0698,9.99,2018-03-14 11:49:36
293644,1649565,527096,0,3,APP0698,9.99,2018-03-14 11:54:35


In [8]:
# orders –> Keep only orders present in orderlines.
orderlines_order_ids = orderlines_cl["id_order"]
orders_cl = orders_cl[orders_cl["order_id"].isin(orderlines_order_ids)].copy()

In [62]:
# checking if that worked
orders_cl["order_id"].nunique() == orderlines_cl["id_order"].nunique()

True

In [64]:
# Exclude orders with unknown products
intersection = set(orders_cl.order_id).intersection(set(orderlines_cl.id_order))
orders_cl = orders_cl.loc[orders_cl.order_id.isin(intersection),:]

In [65]:
orders_cl["order_id"].nunique() == orderlines_cl.id_order.nunique()

True

In [18]:
# Exploring the revenue from different tables
orderlines_cl.loc[:,"unit_price_total"] = orderlines_cl.loc[:,"unit_price"] * orderlines_cl.loc[:,"product_quantity"]

In [20]:
# sum of unit_price_total
ol_grouped = orderlines_cl.groupby("id_order")["unit_price_total"].sum()

In [21]:
ol_grouped

id_order
241423   129.16
242832    10.77
243330    77.99
245275   149.00
245595   105.98
          ...  
527042    13.99
527070    19.98
527074    19.98
527096    29.97
527112     9.99
Name: unit_price_total, Length: 45379, dtype: float64

In [22]:
# What is the average difference between total_paid and unit_price_total?
orders_diff = orders_cl.merge(ol_grouped, how="left", left_on="order_id", right_on="id_order")

In [23]:
# What is the average difference between total_paid and unit_price_total?
orders_diff.loc[:,"diff"] = orders_diff.loc[:,"total_paid"] - orders_diff.loc[:,"unit_price_total"]
orders_diff.loc[:,"diff"].mean()

4.267111659578219

In [24]:
# What is the distribution of these differences?
orders_diff["diff"].describe()

count   45379.00
mean        4.27
std        25.03
min      -165.00
25%         0.00
50%         4.99
75%         6.99
max      3984.99
Name: diff, dtype: float64

In [26]:
# kicking out outliers by quartiles
# calculate the quartiles
Q1 = orders_diff["diff"].quantile(0.25)
Q3 = orders_diff["diff"].quantile(0.75)

# calculate the interquartile range
IQR = Q3-Q1

# filter the DataFrame to include only "non-outliers"
diff_no_outliers_df = orders_diff.loc[(orders_diff["diff"] >= (Q1 - 1.5*IQR)) & (orders_diff["diff"] <= (Q3 + 1.5*IQR))]

In [27]:
diff_no_outliers_df.describe()

,order_id,created_date,total_paid,unit_price_total,diff
count,44651.00,44651,44651.00,44651.00,44651.00
mean,415140.18,2017-09-12 14:29:35.345031424,333.07,329.31,3.76
min,241423.00,2017-01-01 01:51:47,2.82,2.49,-5.00
25%,362869.50,2017-06-10 20:26:20,49.98,45.88,0.00
50%,418045.00,2017-10-28 13:26:36,109.80,104.99,4.99
75%,470832.50,2017-12-26 00:17:39.500000,322.99,319.00,6.99
max,527112.00,2018-03-14 12:03:52,13387.78,13387.77,16.97
std,65716.27,NaN,618.06,618.86,2.91


In [28]:
# now we remove this from the original tables both orders and orderlines

normal_diff_list = list(diff_no_outliers_df["order_id"])

In [68]:
orderlines_cl = orderlines_cl.loc[orderlines_cl["id_order"].isin(normal_diff_list), :]
orders_cl = orders_diff.loc[orders_diff["order_id"].isin(normal_diff_list), :]
orderlines_cl.id_order.nunique() == orders_cl.order_id.nunique()

True

In [69]:
orders_cl["diff"] # these are the shipping costs

0       6.99
1       4.99
2       6.99
3       0.00
4       6.99
        ... 
45374   4.99
45375   4.99
45376   4.99
45377   4.99
45378   4.99
Name: diff, Length: 44651, dtype: float64

In [37]:
# adding categories to the products table using both regex and using some of the information in corrupted column type

In [38]:
category_type_df = products_cl.copy()

In [ ]:
# after exploring the column type: the last 4 digits always explain a broader category (eg iPhones).
# When there is more than 4 digits, the numbers in front explain a subcategory (eg iPhone 6)
# I therefore will only work with the last 4 digits of each string.

In [41]:
# fixing the type column by replacing exponentials
category_type_df.loc[category_type_df["type"].str.contains("\+", na=False), "type"] = category_type_df["type"].str.replace(",", ".")
category_type_df.loc[category_type_df["type"].str.contains("\+", na=False), "type"] = category_type_df["type"].str.replace("E+", "e")
category_type_df.loc[category_type_df["type"].str.contains("\+", na=False), "type"] = category_type_df["type"].str.replace("\+", "")

In [43]:
category_type_df["type"] = category_type_df["type"].astype(float)

In [45]:
category_type_df["type"] = category_type_df["type"].fillna(0)

In [46]:
category_type_df["type"] = category_type_df["type"].astype(int)

In [47]:
category_type_df["type"] = category_type_df["type"].astype(str)

In [48]:
category_type_df["short_type"] = category_type_df["type"].str[-4:]

In [49]:
category_type_df["short_type"].unique()

array(['8696', '5401', '1387', '1230', '1364', '1325', '5384', '1334',
       '5399', '5403', '5397', '5395', '1216', '5400', '0000', '1276',
       '5404', '1296', '1229', '1404', '1405', '4259', '1282', '1424',
       '9094', '0', '5406', '0142', '0230', '1715', '5405', '5398',
       '5407', '2062', '1280', '1433', '1515', '5720', '1298', '1392',
       '1231', '1375', '1714', '1416', '1434', '2425', '2434', '2449',
       '1716', '5185', '2158', '1859', '2282'], dtype=object)

In [50]:
category_type_df[category_type_df.type=="0"] # 47 missing values

,sku,name,desc,price,promo_price,in_stock,type,decimals,len,fixed_promo,new,new_promo,discount,short_type
index,,,,,,,,,,,,,,
307,SAN0017,SanDisk Cruzer Edge USB 2.0 Flash Drive 16GB,Pendrive USB 2.0 Mac and PC.,8.00,5.99,0,0,8,1,59895,5.99,5.99,2.01,0
530,SAN0026,SanDisk Extreme Cruzer 16GB USB 3.0 Flash Drive,USB 3.0 flash drive 16GB USB Mac and PC.,22.00,20.99,0,0,22,2,209935,20.99,20.99,1.01,0
798,APP0675,Apple iPhone 5S 32GB Space Gray,New Free iPhone 5S 32GB (ME435Y / A).,559.00,399.00,0,0,559,3,3989975,399.00,399.00,160.00,0
1193,APP0823,Apple iPhone 6 16GB Silver,New iPhone 6 16GB Free (MG482QL / A).,639.00,639.00,0,0,639,3,639001,639.00,639.00,0.00,0
1199,APP0829,Apple iPhone 6 Plus 16GB Silver,New iPhone 6 Plus 16G Free (MGA92QL / A).,749.00,749.00,0,0,749,3,7490021,749.00,749.00,0.00,0
1200,APP0822,Apple iPhone 6 16GB Space Gray,New iPhone 6 16GB Free (MG472QL / A).,639.00,639.00,0,0,639,3,639001,639.00,639.00,0.00,0
1201,APP0825,Apple iPhone 6 64GB Space Gray,New iPhone 6 64GB Free (MG4F2QL / A).,749.00,749.00,0,0,749,3,7490021,749.00,749.00,0.00,0
1202,APP0826,Apple iPhone 6 64GB Silver,New iPhone 6 64GB Free (MG4H2QL / A).,749.00,750.33,0,0,749,3,7503331,750.33,750.33,-1.33,0
1203,APP0828,Apple iPhone 6 Plus 16GB Space Gray,New 16GB iPhone 6 Plus Free (MGA82QL / A).,749.00,749.00,0,0,749,3,7490021,749.00,749.00,0.00,0


In [51]:
category_type_df.loc[category_type_df.type.str.contains("000"), "short_type"] = category_type_df["type"].str[:4]

In [54]:
category_type_df["short_type"] = category_type_df["short_type"].astype(int)

In [70]:
category_type_df["short_type"].nunique() # still 64 remaining categories

64

In [59]:
# most sold categories sorted after type
category_type_df.groupby("short_type").count().nlargest(10, "sku")

,sku,name,desc,price,promo_price,in_stock,type,decimals,len,fixed_promo,new,new_promo,discount
short_type,,,,,,,,,,,,,
5397,2060,2060,2059,2060,2060,2060,2060,2060,2060,2060,1878,1878,2060
5403,1835,1835,1835,1835,1835,1835,1835,1835,1835,1835,1582,1582,1835
1298,820,820,818,820,820,820,820,820,820,820,697,697,820
5404,481,481,480,481,481,481,481,481,481,481,410,410,481
1282,373,373,373,373,373,373,373,373,373,373,353,353,373
5395,276,276,276,276,276,276,276,276,276,276,255,255,276
5740,247,247,247,247,247,247,247,247,247,247,240,240,247
2158,229,229,229,229,229,229,229,229,229,229,228,228,229
1716,221,221,221,221,221,221,221,221,221,221,190,190,221


In [71]:
category_type_df[category_type_df["short_type"] == 5403] # cases and protectors

,sku,name,desc,price,promo_price,in_stock,type,decimals,len,fixed_promo,new,new_promo,discount,short_type
index,,,,,,,,,,,,,,
15,MOS0021,Clearguard Moshi MacBook Pro and Air,Keyboard Protector MacBook Pro 13-inch Retina ...,24.95,24.99,0,13835403,24,2,249889,24.99,24.99,-0.04,5403
18,MUV0009,Muvit Back Clear Case iPhone 4 / 4S Transparent,Case iPhone 4 / 4S polycarbonate.,9.99,7.99,0,11865403,9,1,7986,NaN,NaN,2.00,5403
37,GRT0196,Griffin Survivor Extreme-Duty Case iPad 2 3 an...,IPad 2 Case 3 and 4 for extreme ultra resistant.,64.99,49.99,0,12635403,64,2,499899,49.99,49.99,15.00,5403
38,MUV0023,Extreme Muvit iPhone and iPod BikeMount sleeve...,IPhone and iPod support sleeve waterproof App ...,35.90,16.99,0,11865403,35,2,169896,16.99,16.99,18.91,5403
46,TWS0026,"Twelve South BookBook MacBook Air sleeve 11 ""b...",Case Macbook 11 inches vintage skin with soft ...,79.99,75.00,0,13835403,79,2,749958,75.00,75.00,4.99,5403
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19279,MMW0012,"My MW Case MacBook Pro 13 ""(Late 2016) Blue Sa...",Avoid shock and damage to your MacBook Pro 13-...,29.99,20.99,0,13835403,29,2,2099,NaN,NaN,9.00,5403
19280,MMW0013,"My MW Case MacBook Pro 13 ""(Late 2016) White S...",Avoid shock and damage to your MacBook Pro 13-...,29.99,20.99,1,13835403,29,2,2099,NaN,NaN,9.00,5403
19281,MMW0016,"My MW Case MacBook Pro 13 ""(Late 2016) Black",Avoid shock and damage to your MacBook Pro 13-...,29.99,20.99,1,13835403,29,2,2099,NaN,NaN,9.00,5403


In [72]:
# adding a column for the short brand name
category_type_df["brand_short"] = category_type_df["sku"].str[:3]

In [73]:
# cleaning the unnecessary columns
category_type_df.drop(columns=["decimals", "len", "fixed_promo", "new", "new_promo"],inplace=True)

In [74]:
# we will keep working with these categories in the future.
category_type_df["category"] = "other"
category_type_df.loc[category_type_df['desc'].str.contains('chil|kid|toy|game', case=False), "category"] = "games/toys"
category_type_df.loc[category_type_df['desc'].str.contains('fit|cardio|sport|activ|Sphygmomanometer|measuring|run|health', case=False), "category"] = "sport"
category_type_df.loc[category_type_df['desc'].str.contains('music|loud|microph|head|dj|cd|dvd|speaker', case=False), "category"] = "music"
category_type_df.loc[category_type_df['desc'].str.contains('charge|adapter|cable|wire|batter|power|connect|transformer|protector|usb|cable', case=False), "category"] = "power"
category_type_df.loc[category_type_df['name'].str.contains('case', case=False), "category"] = "cases"
category_type_df.loc[category_type_df[('desc')].str.contains('care|protect', case=False), "category"] = "care"
category_type_df.loc[category_type_df[('name')].str.contains('care|protect', case=False), "category"] = "care"
category_type_df.loc[category_type_df['name'].str.contains('keyboard|mouse|pen|Toolkit', case=False), "category"] = "computer accessories"
category_type_df.loc[category_type_df['name'].str.startswith('Apple Macbook'), "category"] = 'Macbook'
category_type_df.loc[category_type_df['name'].str.startswith('Apple iPhone 8|Apple iPhone 7|Apple iPhone SE|Apple iPhone 3|Apple iPhone 6|Apple iPhone 7'), "category"] = 'iPhone'
category_type_df.loc[category_type_df['name'].str.contains('watch', case=False), "category"] = "watch"
category_type_df.loc[(category_type_df['sku'].str.startswith('AP')) & (category_type_df['price'] > 300), 'category'] = "iPad/Mac/Tablet"
category_type_df.loc[category_type_df['name'].str.startswith('Apple iMac 2'),'category'] = "iPad/Mac/Tablet"
category_type_df.loc[category_type_df['name'].str.contains('tablet', case=False), "category"] = "iPad/Mac/Tablet"
category_type_df.loc[(category_type_df['desc'].str.contains('server')) & (category_type_df['name'].str.contains('server')), "category"] = "server"
category_type_df.loc[category_type_df['desc'].str.contains('monitor|display', case=False), "category"] = "monitor"
category_type_df.loc[category_type_df['desc'].str.contains('memory|drive', case=False), "category"] = "memory"
category_type_df.loc[category_type_df['desc'].str.contains('router|satellite', case=False), "category"] = "routers"
category_type_df.loc[category_type_df['desc'].str.contains('camera', case=False), "category"] = "cameras"
category_type_df.loc[category_type_df['desc'].str.contains('bag|Backpack', case=False), "category"] = 'bags'
category_type_df.loc[category_type_df['desc'].str.contains('expansion', case=False), "category"] = 'expansion'

ValueError: Cannot mask with non-boolean array containing NA / NaN values

In [75]:
# adding this category defined by short_type
category_type_df.loc[category_type_df["short_type"]==1716, "category"] = "iPhone"

In [76]:
# renaming columns
category_type_df.rename(columns = {"long":"brand"}, inplace=True)

In [77]:
category_type_df["sku"].nunique(), orderlines_cl["sku"].nunique() # not all products on products table have been sold


(10157, 5808)

In [ ]:
# adding a discount column
category_name_df["discount"] = category_name_df["price"] - category_name_df["promo_price"]

In [79]:
orderlines_cl["id_order"].nunique() == orders_cl["order_id"].nunique() 

True

In [ ]:
category_type_df.to_parquet("products_qu.parquet")
orders_cl.to_csv("orders_qu.csv")
orderlines_cl.to_parquet("orderlines_qu.parquet")